In [0]:
from dotenv import load_dotenv
import os
from typing import Literal, TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState

# Initialize LLM
llm = ChatDatabricks(endpoint="databricks-gpt-oss-120b")

def call_llm(state: MessagesState):
    """Node to call the LLM with a formatted prompt based on the latest user question."""
    
    
    # Build the prompt
    prompt_template = (
        "You are an expert AI assistant. Answer the user's question with clarity, accuracy, and conciseness.\n\n"
        "## Question:\n"
        "{question}\n\n"
        "## Guidelines:\n"
        "- Keep responses factual and to the point.\n"
        "- If relevant, provide examples or step-by-step instructions.\n"
        "- If the question is ambiguous, clarify before answering.\n\n"
        "Respond below:"
    )
    formatted_prompt = prompt_template.format(question=state['messages'])

    # Construct the message list for the model
    all_msgs = [
        SystemMessage(content="You are an expert AI assistant."),
        HumanMessage(content=formatted_prompt)
    ]

    # Call the LLM
    response = llm.invoke(all_msgs)

    return {"messages": state["messages"] + [response]}

# Build the graph
builder = StateGraph(MessagesState)
builder.add_node("call_llm", call_llm)

builder.add_edge(START, "call_llm")
builder.add_edge("call_llm", END)

graph = builder.compile()

# Example run
messages = graph.invoke({
    "messages": [HumanMessage(content="What is the return policy for Anirvan Decodes ecommerce products?")]
})


In [0]:
message = messages['messages'][-1]

for part in message.content:
    if part.get("type") == "text":
       ai_message = part.get("text", "")

print(ai_message)